<a href="https://colab.research.google.com/github/cedjustin/AI/blob/master/sprint13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
a = tf.constant(5)
b = tf.constant(7)
add = tf.add(a,b)
sess = tf.compat.v1.Session()
output = sess.run(add)
print(output) 
sess.close()

12


In [ ]:
with tf.compat.v1.Session() as sess:
  output = sess.add(a,b)
  print(output)